In [ ]:
#call library

import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input, RepeatVector,GRU
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import ta
import json
#from preprocesslib import preprocessEach, makeY

In [ ]:
import os
import pandas as pd
import numpy as np

'''
이런식으로 사용하면 됩니다.
from preprocesslib import preprocessEach, makeY
X = preprocessEach('overlap')
X bootstrap해서 -> X_final만든 후에
y = makeY('overlap', len(X_final))
'''

def preprocessEach(folder_name):
   # set file path
    # folder_path = f'/content/drive/MyDrive/research/data/{folder_name}_clean'
    folder_path = f'../data/{folder_name}_clean'


    # load file name in folder
    file_names = os.listdir(folder_path)
    data_array = []

    for file in file_names:
        # set file path
        file_path = os.path.join(folder_path, file)
        
        # load csv file as dataframe
        df = pd.read_csv(file_path, index_col=0)
        df.index = pd.to_datetime(df.index)

        #preprocessing column
        df['open-close'] = df['Open'] - df['Close'] #추가
        df['open-close'] = df['open-close'] + abs(min(df['open-close'])) #추가
        df['MA5'] = df['Close'].rolling(5).mean() #추가
        df['Diff'] = df['High'] - df['Low']
        df['RSI'] = ta.momentum.rsi(df['Close'])
        df['MACD'] = ta.trend.macd(df['Close'])


        df = df[['Close','Volume','Diff','open-close','MA5', 'Search','MACD','RSI']]
        df.fillna(0,inplace=True)



        #log transform
        df = np.log1p(df)
        # df.reset_index(inplace=True)
        df.fillna(0,inplace=True)

        #insert list to array
        data_array.append(df)

    return np.array(data_array)

def makeYValue(folder_name, filelength):
    if folder_name=='growth':
        yValues = np.zeros(filelength)
    elif folder_name=='value':
        yValues = np.ones(filelength)

    y = np.column_stack([yValues.T])
    return y

def bootstrap(large, small):
    diff = len(large) - len(small)
    bootstrapped = small[np.random.choice(len(small), size=diff, replace=True)]
    return np.concatenate((small, bootstrapped), axis=0)

In [ ]:
xGrowths = preprocessEach('growth')
xValues = preprocessEach('value')
xOverlaps = preprocessEach('overlap')

In [ ]:
xValues = np.concatenate((xValues, xOverlaps), axis=0)

In [ ]:
xGrowths = bootstrap(xValues, xGrowths)

In [ ]:
#make y values
yGrowths = makeYValue('growth', len(xGrowths))
yValues = makeYValue('value', len(xValues))

In [ ]:
x = np.concatenate((xGrowths, xValues), axis=0)
y = np.concatenate((yGrowths, yValues), axis=0)

In [ ]:
#minmax scaling by element
scaler = MinMaxScaler()
for i in range(8):
    x[:,:,i] = scaler.fit_transform(x[:,:,i])

In [ ]:
def splitData(x, y, train_ratio=0.8):
    # shuffle data
    np.random.seed(2023)
    permutation = np.random.permutation(len(x))
    x = x[permutation]
    y = y[permutation]

    # split data
    n_train_samples = int(train_ratio * len(x))
    x_train, x_test = x[:n_train_samples], x[n_train_samples:]
    y_train, y_test = y[:n_train_samples], y[n_train_samples:]

    return x_train, x_test, y_train, y_test

In [ ]:
X_train, X_test, Y_train, Y_test = splitData(x,y,0.8)

X_train = np.nan_to_num(X_train, nan=0)
X_test = np.nan_to_num(X_test, nan=0)

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [ ]:
model_encoder=load_model('./dimensionReduction.h5')
X_train_encoded = model_encoder.predict(X_train)

In [ ]:
X_train_encoded.shape

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=300)
mc = ModelCheckpoint('best_model_value.h5', monitor='val_acc', mode='max', save_best_only=True)

def create_model():
    model = Sequential()
    model.add(GRU(64, input_shape=(1248,3), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(16))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [ ]:
x_split, y_split = splitDataCrossVal(X_train_encoded, Y_train)

In [ ]:
print(len(x_split))
accuracy_cross_val = []
for i in range(len(x_split)):
  x_train, x_test, y_train, y_test = cvsplitData(x_split, y_split, i)
  x_train = np.nan_to_num(x_train, nan=0)
  x_test = np.nan_to_num(x_test, nan=0)

  model = create_model()

  history=model.fit(x_train, y_train, epochs=2000, validation_data=(x_test, y_test), callbacks=[es, mc])
  model=load_model('best_model_value.h5')
  loss, accuracy = model.evaluate(x_test, y_test)
  accuracy_cross_val.append(accuracy)
  with open(f'history_value{i}.json', 'w') as f:
      json.dump(history.history, f)
print("cross validation accuracy:{}".format(accuracy_cross_val))
print("average cross validation accuracy:{}".format(sum(accuracy_cross_val)/len(accuracy_cross_val)))

In [ ]:
count_01 = np.sum((y_test == [0,1]).all(axis=1))
count_10 = np.sum((y_test == [1,0]).all(axis=1))

print(f"[01]의 개수: {count_01}")
print(f"[10]의 개수: {count_10}")

In [ ]:
best_model=load_model('best_model_value.h5')
loss, accuracy = best_model.evaluate(x_test, y_test)
y_pred_prob = best_model.predict(x_test)

In [ ]:
hiostory_dict_list = []
for i in range(len(x_split)):
    with open(f'history_value{i}.json', 'r') as f:
        hiostory_dict_list.append(json.load(f))

In [ ]:
def model_loss(history):
    epochs = range(1, len(history['val_acc']) + 1)
    plt.plot(epochs, history['loss'])
    plt.plot(epochs, history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
def model_acc(history):
    epochs = range(1, len(history['val_acc']) + 1)
    plt.plot(epochs, history['acc'])
    plt.plot(epochs, history['val_acc'])
    plt.title('model acc')
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
for i in hiostory_dict_list:
    model_loss(i)
    model_acc(i)

In [ ]:
mc_final = ModelCheckpoint('best_model_value_final.h5', monitor='val_acc', mode='max', save_best_only=True)

In [ ]:
final_model = create_model()

In [ ]:
history_final=final_model.fit(X_train_encoded, Y_train, epochs=2000, validation_split=0.2, callbacks=[es, mc_final])

In [ ]:
with open('history_value_final.json', 'w') as f:
    json.dump(history_final.history, f)

In [ ]:
model_encoder=load_model('./dimensionReduction.h5')
X_test_encoded = model_encoder.predict(X_test)

In [ ]:
best_model_final=load_model('best_model_value_final.h5')

In [ ]:
loss_f, accuracy_f = best_model_final.evaluate(X_test_encoded, Y_test)
y_pred_prob_f = best_model_final.predict(X_test_encoded)